In [47]:
from surprise import Dataset, Reader
import pandas as pd
from surprise import Trainset
from surprise import SVD
from surprise.model_selection import cross_validate
import gradio as gr

In [48]:
# cargamos los datos en un dataframe de pandas
df = pd.read_csv('./DATOS/RATINGs_PARA_ML.csv')

In [49]:

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'title', 'rating_y']][:800000], reader)


In [50]:

trainset = data.build_full_trainset()


In [51]:
#utilizar el algoritmo de recomendación SVD

model = SVD()
model.fit(trainset)


In [52]:
#Para evaluar el rendimiento del modelo, se puede utilizar la función cross_validatede Surprise:
results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0108  1.0125  1.0080  1.0139  1.0134  1.0117  0.0021  
MAE (testset)     0.7920  0.7931  0.7899  0.7936  0.7933  0.7924  0.0013  
Fit time          6.43    6.49    6.46    6.49    6.47    6.47    0.02    
Test time         1.07    1.42    1.06    1.39    1.02    1.19    0.17    


In [53]:
print('RMSE:', results['test_rmse'].mean())
print('MAE:', results['test_mae'].mean())

RMSE: 1.0117435466758944
MAE: 0.7923879629086148


In [54]:
'''se puede utilizar el método predictpara obtener una predicción del puntaje 
que un usuario le daría a una película.
Para determinar si se debe recomendar o no la película a un usuario en particular,
 se puede establecer un umbral y recomendar películas con una predicción de puntaje por 
 encima de ese umbral.
'''
userId = 582
title = 'monster make'

predicted_rating = model.predict(userId, title)
print(predicted_rating)
threshold = 3.5

if predicted_rating.est > threshold:
    print('Recomendar')
else:
    print('No recomendar')

user: 582        item: monster make r_ui = None   est = 3.64   {'was_impossible': False}
Recomendar


In [46]:
from surprise.model_selection import cross_validate
import numpy as np

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0035  1.0059  1.0059  1.0051  0.0011  
Fit time          1.73    1.71    1.76    1.73    0.02    
Test time         0.86    1.07    0.80    0.91    0.11    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0034  1.0086  1.0017  1.0046  0.0030  
Fit time          1.74    1.68    1.76    1.73    0.03    
Test time         0.80    0.81    0.78    0.80    0.01    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0044  1.0052  1.0067  1.0054  0.0009  
Fit time          1.79    1.71    1.78    1.76    0.04    
Test time         0.84    0.80    0.81    0.82    0.02    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0097  1.0

In [55]:

#GENERAMOS LA FUNCION PARA PONER A PRUEBA EL MODELO

def movie_recommendation(userId, title):
     
    # Obtener la predicción del puntaje para la película dada
    predicted_rating = model.predict(userId, title)

    # Establecer un umbral y recomendar la película si la predicción del puntaje es mayor que el umbral
    threshold = 3.52
    if predicted_rating.est > threshold:
        return 'Recomendar'
    else:
        return 'No recomendar'

In [81]:



demo = gr.Interface(
    fn=movie_recommendation,
    inputs=["number", "text"],
    outputs=["text"],
    )

demo.launch(share=True)



Running on local URL:  http://127.0.0.1:7873
Running on public URL: https://0c7d2527b6bb2d4201.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
